In [1]:
import pyforest
data_path = '/Users/jayceepang/msse/ATOM_CODE/original_datasets/' 

In [123]:
def check_labels(datapath): 
  """a function to check that the labels are indeed what we want (a % binding or % inhibition activiy is 
  >= 50% for that molecule to be active against that NEK)"""
   assay_pct_dfs = [file for file in os.listdir(datapath) if file.startswith('NEK')]
   for file in assay_pct_dfs:
    full_file = os.path.join(datapath, file)
    df = pd.read_csv(full_file)
    
    pct_col = [col for col in df.columns if col.startswith('pct_')] 
    len(pct_col) == 1 
    pct_col = pct_col[0]
    invalid_labels = df[df['active']!= (df[pct_col]>=50).astype(int)]
    if invalid_labels.empty: 
      # print(f'correct label assignment based on {pct_col} column')
      pass
    else: 
      print(f'Error on label assignement.')
      print(invalid_labels)
    # check for duplicates here 
    # duplicates = df[df.duplicated()]
    print(f'{df.shape}')
    # duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
    duplicates = df.duplicated()
    # duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
    if duplicates.any():
        print('Duplicate rows found:')
        print(duplicates)
        df_cleaned = df.drop_duplicates()
        print(f'df size original: {df.shape}')
        print(f'remove duplicates. {df_cleaned.shape}')
check_labels(data_path)

IndentationError: unexpected indent (959015697.py, line 4)

In [128]:
# must specify filename because the 50pct binding and inhibition files differ
# to-do: discuss how to do this --> this will create a file that is just this but with column specifying split
#       do we want to already integrate a split_uuid like AMPL? 
#       additionally, we still have to use the splits to create the "scaled df" 
def remove_duplicates(datapath, filename):
    df = pd.read_csv(f'{datapath}{filename}')
    duplicates = df.duplicated(subset=['base_rdkit_smiles'])
    if duplicates.any():
        print('Duplicate rows found:')
        print(df[duplicates])
        df_cleaned = df.drop_duplicates(subset=['base_rdkit_smiles'])
        print(f'df size original: {df.shape}')
        print(f'Removed duplicates. New df size: {df_cleaned.shape}')
    else: 
        df_cleaned = df
    return df_cleaned
    
def split_data(datapath, filename, train_ratio=.8, test_ratio=0.2): 
    """filename: NEK#_1_uM_min_50_pct_(binding or inhibition).csv"""
    df = remove_duplicates(datapath, filename)

    # is this extra/too much and should we just assume what labels coincide with majority and minority? 
    # determine majority/minority class 
    #       AMPL takes in 'response column' 
    class_labels = df['active'].value_counts() 
    print(class_labels)
    if len(class_labels)>1: 
        majority_class_label =class_labels.idxmax() 
        majority_num = class_labels.max() 
        minority_class_label = class_labels.idxmin()
        minority_count = class_labels.min()
    df_majority = df[df['active']==majority_class_label]
    df_minority=df[df['active']==minority_class_label]
    # copy to avoid warnings 
    df_majority = df_majority.copy()
    df_minority = df_minority.copy()
    n = round(1/test_ratio) # how else can we do this? i think we should keep kfold splits 
            # 1/.2 = 5 splits 
    kf = KFold(n_splits=n,shuffle=True, random_state=42)
    # majority 
    for i, (_, v_ind) in enumerate(kf.split(df_majority)):
        df_majority.loc[df_majority.index[v_ind], 'fold'] = f"fold{i+1}"
    # minority 
    for i, (_, v_ind) in enumerate(kf.split(df_minority)):
        df_minority.loc[df_minority.index[v_ind], 'fold'] = f"fold{i+1}"
    all_fold_df = pd.concat([df_majority,df_minority])
    
    print(all_fold_df.shape)
    print(all_fold_df.active.value_counts())
    ## actually, this might be perfect jsut to save as "split df" because then 
    # in the next step of creating a dataset, you can choose what fold you want to be the test set 
    return all_fold_df
        
    

# check for duplicates and split and save splits

In [ ]:
assay_pct_dfs = [file for file in os.listdir(data_path) if file.startswith('NEK')]

def get_bind_inhib(df_name):
    match = re.search(r'_([^_]+)\.csv$',df_name)
    return match.group(1) if match else None

split_dir = '/Users/jayceepang/msse/ATOM_CODE/datasets/'

for i in range(len(assay_pct_dfs)): 
    bind_inhib = get_bind_inhib(assay_pct_dfs[i])
    nek_num = str(assay_pct_dfs[i][3])
    print(assay_pct_dfs[i])
    NEK_name = f'NEK{nek_num}_{bind_inhib}'
    print(NEK_name)
    cleaned_split_df = split_data(data_path, assay_pct_dfs[i])

    # DO WE DO THIS HERE???  
    cleaned_split_df['NEK'] =NEK_name 
    cleaned_split_df.to_csv(f'{split_dir}{NEK_name}_split.csv',index=False)
    


# Featurization
should we do this per dataset? (intend to run this in a for loop?)

In [ ]:
def scale_dataset(train, test): 
    # file is the split file (folds)
    # add code to do standard scaler 
    # code to scale then concat 
    scaled_df = pd.concat([train,test])
    return scaled_df

def featurize(datapath, filename, feat_type, test_fold = 1, n_bits=None): 
    # if feat_type == 'MOE': 
        # direct to descriptors files and matchup our split df with the features 
    # if feat_type == 'MFP' or 'ECFP': 
        # create MFP feats? 
        # or should we use ECFP? 
        # specify bits (is not None)  
    # 1. split data based on fold 
    df = pd.read_csv(datapath+filename) # this is the split.csv 
    test = df[df['fold']==test_fold]
    train=df[df['fold']!=test_fold] 
    #### HOW DO WE ACCOUNT FOR VALIDATION? ### 
    # 2. standard scaler 
    scaled_df = scale_dataset(train,test)
    # 3. save scaled_df NO MATTER WHAT.
    # ## MAYBE WE SHOULD SAVE IT IN THE SCALE DF FUNCTION?  ## 
    # 4. perform featurization 
    # 5. save datasets 
    # 6. possibly, call function to prep for training (save trainX.csv, train_y.csv, testX.csv, test_y.csv)
    



    

In [137]:
# # I'm checking the scaled_descriptors files because I knew there were duplicates in these files 
# desc_dir = '/Users/jayceepang/msse/ATOM_CODE/original_datasets/scaled_descriptors/'
# desc_dfs = [file for file in os.listdir(desc_dir) if file.startswith('NEK')]

# print(desc_dfs)
# for i in range(len(desc_dfs)): 
#     split_data(desc_dir, desc_dfs[i])

In [60]:
1904+140=2044

2044